<a href="https://colab.research.google.com/github/sruthib22/DS601_Fall21/blob/main/assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Lab
Using the flights dataframe answer the following questions

When do the first and last flights leave each day?

When there is a missing value in dep_time then this is an indication of canceled flight. Find the number of cancelled flights for each (carrier, origin) combination.

Which carrier has the worst dep_delays?

Which plane (tailnum) has the worst on-time record?

For each plane, count the number of flights before the first delay of greater than 1 hour.

In [ ]:
import pandas as pd


url = 'https://raw.githubusercontent.com/mguner/teaching/main/datasets/nyc_flights/nyc_flights.csv'

## read the file
df= pd.read_csv(url)

In [ ]:
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [ ]:
df =df.sort_values('time_hour')

In [ ]:
from datetime import datetime 
df['time_hour'] = pd.to_datetime(df['time_hour'])

In [ ]:
df

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 10:00:00+00:00
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01 10:00:00+00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 10:00:00+00:00
15,2013,1,1,559.0,559,0.0,702.0,706,-4.0,B6,1806,N708JB,JFK,BOS,44.0,187,5,59,2013-01-01 10:00:00+00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 10:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111278,2013,12,31,2355.0,2359,-4.0,430.0,440,-10.0,B6,1503,N509JB,JFK,SJU,195.0,1598,23,59,2014-01-01 04:00:00+00:00
111279,2013,12,31,2356.0,2359,-3.0,436.0,445,-9.0,B6,745,N665JB,JFK,PSE,200.0,1617,23,59,2014-01-01 04:00:00+00:00
111276,2013,12,31,2328.0,2330,-2.0,412.0,409,3.0,B6,1389,N651JB,EWR,SJU,198.0,1608,23,30,2014-01-01 04:00:00+00:00
110520,2013,12,31,13.0,2359,14.0,439.0,437,2.0,B6,839,N566JB,JFK,BQN,189.0,1576,23,59,2014-01-01 04:00:00+00:00


### 1. when do the first and last flight leave

In [ ]:
dep_flight = df.groupby(['carrier','day']).agg(
    First_fgt = ('dep_time', 'min') , 
    Last_Fgt = ('dep_time', 'max'))

dep_flight

First_fgt  Last_Fgt
carrier day                     
9E      1        600.0    2349.0
        2        600.0    2250.0
        3        554.0    2312.0
        4        557.0    2216.0
        5         16.0    2157.0
...                ...       ...
YV      27      1147.0    2319.0
        28      1131.0    1930.0
        29      1134.0    1840.0
        30      1143.0    2000.0
        31      1125.0    1905.0

[490 rows x 2 columns]

In [ ]:
#res = df.groupby(df.time_hour).apply(lambda x: x.iloc[[0, -1]])
#res.index = res.index.droplevel(0)

#print(res)

### 2. canccelled flights

In [ ]:
df.groupby('day').dep_time.min()

day
1     1.0
2     3.0
3     2.0
4     3.0
5     1.0
6     4.0
7     2.0
8     1.0
9     2.0
10    1.0
11    1.0
12    2.0
13    1.0
14    3.0
15    1.0
16    1.0
17    2.0
18    1.0
19    1.0
20    1.0
21    2.0
22    1.0
23    4.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    2.0
30    1.0
31    1.0
Name: dep_time, dtype: float64

### 3. carrier which has the worst dep delays

In [ ]:
df['dep_time'].isnull().value_counts()

False    328521
True       8255
Name: dep_time, dtype: int64

In [ ]:
cancel_flight= df.groupby(['origin','carrier']).agg({'dep_time':'count'})
cancel_flight

dep_time
origin carrier          
EWR    9E           1200
       AA           3388
       AS            712
       B6           6483
       DL           4303
       EV          41775
       MQ           2108
       OO              6
       UA          45652
       US           4330
       VX           1556
       WN           6083
JFK    9E          13844
       AA          13642
       B6          41761
       DL          20601
       EV           1326
       HA            342
       MQ           6866
       UA           4490
       US           2969
       VX           3575
LGA    9E           2372
       AA          15063
       B6           5925
       DL          22857
       EV           8255
       F9            682
       FL           3187
       MQ          16189
       OO             23
       UA           7837
       US          12574
       WN           6000
       YV            545

In [ ]:
worst_flights = df.groupby(["carrier"]).agg({'dep_delay':'max'})
worst_flights

,dep_delay
carrier,
9E,747.0
AA,1014.0
AS,225.0
B6,502.0
DL,960.0
EV,548.0
F9,853.0
FL,602.0
HA,1301.0


### 4.worst on-time record

In [ ]:
worst_on_time = df.groupby(['tailnum']).agg(max_delay = ('dep_delay', 'max'))

max_delay = worst_on_time['max_delay'].max()

worst_on_time[worst_on_time['max_delay'] == max_delay]

,max_delay
tailnum,
N384HA,1301.0


### 5. for each plane count the number of delays

In [ ]:
flight_delay_count = df.groupby(['tailnum', 'dep_delay']).agg(Fght_Cnt = ('flight', 'count'),
                                                              Dept_delay = ('dep_delay', 'max'),
                                                              Tailnum = ('tailnum', 'max'))


flight_delay_count

Fght_Cnt  Dept_delay Tailnum
tailnum dep_delay                              
D942DN  -6.0              1        -6.0  D942DN
         24.0             1        24.0  D942DN
         40.0             1        40.0  D942DN
         68.0             1        68.0  D942DN
N0EGMQ  -15.0             4       -15.0  N0EGMQ
...                     ...         ...     ...
N9EAMQ   143.0            1       143.0  N9EAMQ
         156.0            1       156.0  N9EAMQ
         169.0            1       169.0  N9EAMQ
         184.0            1       184.0  N9EAMQ
         268.0            1       268.0  N9EAMQ

[122675 rows x 3 columns]